In [1]:
import tensorflow as tf
import tflearn
import numpy as np
import tensorflow.examples.tutorials.mnist.input_data as input_data
tf.logging.set_verbosity(tf.logging.ERROR) # 警告を非表示にする

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
# Fashion-MNISTをダウンロードしてdata/fashionに格納
fashion = input_data.read_data_sets(
    'data/fashion', # 格納先のフォルダーを作成
    source_url='http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/',
    one_hot=True    # ラベルをワンホット表現にする
)

Extracting data/fashion\train-images-idx3-ubyte.gz
Extracting data/fashion\train-labels-idx1-ubyte.gz
Extracting data/fashion\t10k-images-idx3-ubyte.gz
Extracting data/fashion\t10k-labels-idx1-ubyte.gz


In [3]:
# 訓練用データをすべて抽出
train_images, train_labels = fashion.train.next_batch(60000)
print(train_images.shape)  #出力：(60000, 784)
print(train_labels.shape)  #出力：(60000, 10)

# テスト用データをすべて抽出
test_images, test_labels = fashion.test.next_batch(10000)
print(test_images.shape)  #出力：(10000, 784)
print(test_labels.shape)  #出力：(10000, 10)

(60000, 784)
(60000, 10)
(10000, 784)
(10000, 10)


In [4]:
# データの前処理
# データの前処理（時系列データとして扱えるようにする）
# 訓練データ(60000, 784)を(60000, 28, 28)に変換
train_images = np.reshape(train_images, (-1, 28, 28))

# テストデータ(10000, 784)を(10000, 28, 28)に変換
test_images = np.reshape(test_images, (-1, 28, 28))

In [5]:
# RNNの構築

## 初期化
tf.reset_default_graph()

## 入力層の作成
net = tflearn.input_data(shape=[None, 28, 28])

## 中間層の作成
# LSTMブロック
net = tflearn.lstm(net, 128, dropout=0.25, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.25, return_seq=True)
net = tflearn.lstm(net, 128, dropout=0.25, return_seq=False)

## 出力層
net = tflearn.fully_connected( # 全結合型の層を配置
    incoming=net,              # 入力として1つ前の層を指定
    n_units=10,                # ニューロン数は10個
    activation='softmax'       # 活性化関数はソフトマックス
)

# 学習条件の設定
net = tflearn.regression(
    incoming=net,                   # 学習の対象となる層としてnetを指定
    optimizer='adam',               # 最適化はAdamアルゴリズムで行う
    learning_rate=0.001,            # 学習率
    loss='categorical_crossentropy' # 誤差関数はクロスエントロピー誤差
)

In [6]:
# モデルの作成と学習の実行
model = tflearn.DNN(net)      # 作成したCNNと学習条件をセット
model.fit(train_images,       # 訓練データ(画像)
          train_labels,       # 訓練データ(ラベル)
          n_epoch=50,         # 学習回数
          batch_size=100,     # ミニバッチのサイズ
          validation_set=None,# 訓練データからテストデータを抽出しない
          show_metric=True    # 各ステップごとに精度を出力
         )

Training Step: 29999  | total loss: 0.53751 | time: 77.193s
| Adam | epoch: 050 | loss: 0.53751 - acc: 0.8107 -- iter: 59900/60000
Training Step: 30000  | total loss: 0.53505 | time: 77.320s
| Adam | epoch: 050 | loss: 0.53505 - acc: 0.8127 -- iter: 60000/60000
--


In [7]:
# テストデータを使用して予測する
pred = np.array(model.predict(test_images)).argmax(axis=1)
print(pred)

# 正解ラベルを取得
label = test_labels.argmax(axis=1)
print(label)

print(pred.shape)
print(label.shape)

# 予測値と正解ラベルが一致している割合（精度）
accuracy = np.mean(pred == label, axis=0)
print(accuracy)

[9 2 9 ... 0 9 7]
[9 2 9 ... 0 5 5]
(10000,)
(10000,)
0.8331
